In [27]:
# from bs4 import BeautifulSoup

# df = pd.read_csv('audiodata_mfa.csv')

# # Metadata
# f = open('corpus/karuk-texts.xml', 'r')
# contents = f.read()
# soup = BeautifulSoup(contents,'xml')

# speakers = []

# for audio in df.audio:
#     try:
#         speakers.append(soup.find("s", {"audio" : audio}).get('spkr'))
#     except AttributeError:
#         speakers.append(None)

# df['speaker'] = speakers

# #df.to_csv('audio_metadata.csv', index = False)

# Read in Karuk data and look for target items

In [1]:
import pandas as pd
import os
import re
import shutil
import glob

In [2]:
df = pd.read_csv('audio_metadata.csv')

### Search for labialization and palatalization examples
ika, iku, upi, ufi, uxi

In [4]:
# Counts of speaker sentences
df['speaker'].value_counts(normalize=True)

VS    0.693721
NR    0.107708
SD    0.042879
LA    0.042879
JS    0.036243
CT    0.026544
BJ    0.020929
CP    0.017356
JL    0.009188
LB    0.002552
Name: speaker, dtype: float64

In [4]:
df_v = df.loc[df['speaker'] == 'VS']

In [12]:
# All labialization and palatalization
#pat = r'.*\d?(i\d?k[au]|u\d?[pfx]i).*'

# Vowels preceded by target C word boundary: fu, pu, xu, ki
# ?! is negative lookahead--searches for NOT a sequence"
# Find sequence that is at beginning of word (\b)
# Then followed by any of the labial beginnings
# Then one or two u's with potential digits
# NOT followed by fpx again with i after
pat = r'.*\b([fpx]u\d?u?\d?(?!u?\d?[fpx]i)|ki\d?i?\d?(?!i?\d?ka))'
folder = 'vowels'
# CAN'T get rid of puufich for some reason so must manually rid

# Vowel + C + same vowel
#pat = r'.*(u\d?u?\d?[fpx]u\d?u?\d?(?!u?\d?[fpx]i)|i\d?i?\d?ki\d?i?\d?(?!i?\d?ka))'
#folder = 'vowels_double_u-i'
# tukeevniikichha is an issue
# keevniikich
# utapkuupu

# Subset dataset with only sentences containing matched patterns
df_match = df_v[df_v['transcription_mfa'].str.match(pat) == True]

# Split sentences into words
all_words = df_match['transcription_mfa'].str.split()

# Gather the words only that match
df_match['matched_words'] = all_words.apply(lambda l: ','.join([word for word in l if re.compile(pat).search(word)]))

# Get sequences that match in each word
seq_list = all_words.apply(lambda l: [re.compile(pat).findall(w) for w in l if re.compile(pat).search(w)])

# Flatten list of lists
import collections

def flatten(l):
    for el in l:
        if isinstance(el, collections.Iterable) and not isinstance(el, (str, bytes)):
            yield from flatten(el)
        else:
            yield el

flat_seq_list = seq_list.apply(lambda seq: list(flatten(seq)))

# Remove empty items and add sequences
df_match['sequences'] = flat_seq_list.apply(lambda s: ','.join([seq for seq in s if seq != '']))

# Print transcriptions
#print(df_match.transcription_mfa)

df_match.to_csv('./VS/' + folder + '/targets.csv', index = False)

# Gather audio files for copying
files = df_v[df_v['transcription_mfa'].str.match(pat) == True].audio

print(len(files))

for f in files:
    
    for x in glob.glob('corpus/force-aligned/*/' + f + '.*', recursive = True):
        
        shutil.copy(x, './VS/' + folder + '/files')

/Users/Raksit/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Raksit/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


158


# Change TextGrids to have target segments on another tier

In [6]:
import audiolabel as al

In [7]:
seq_to_arpa = {
    'pu': ['P', 'UH0'],
    'puu': ['P', 'UW1'],
    'fu': ['F', 'UH0'],
    'fuu': ['F', 'UW1'],
    'xu': ['HH', 'UH0'],
    'xuu': ['HH', 'UW1'],
    'ki': ['K', 'IH0'],
    'kii': ['K', 'IY1'],
    # Double
    'upu': ['UH0', 'P', 'UH0'],
    'uupu': ['UW1', 'P', 'UH0'],
    'upuu': ['UH0', 'P', 'UW1'],
    'uupuu': ['UW1', 'P', 'UW1'],
    'ufu': ['UH0', 'F', 'UH0'],
    'uufu': ['UW1', 'F', 'UH0'],
    'ufuu': ['UH0', 'F', 'UW1'],
    'uufuu': ['UW1', 'F', 'UW1'],
    'uxu': ['UH0', 'HH', 'UH0'],
    'uuxu': ['UW1', 'HH', 'UH0'],
    'uxuu': ['UH0', 'HH', 'UW1'],
    'uuxuu': ['UW1', 'HH', 'UW1'],
    'iki': ['IH0', 'K', 'IH0'],
    'iiki': ['IY1', 'K', 'IH0'],
    'ikii': ['IH0', 'K', 'IY1'],
    'iikii': ['IY1', 'K', 'IY1'],
}

In [8]:
def create_target_tiers(f, metadata):
    '''
    Create tiers for sequences and for phonemes
    '''

    # Read in targets csv
    df = pd.read_csv(metadata)
    
    fnoext = f.split('/')[-1].split('.')[0]
    
    words = df[df.audio == fnoext].matched_words.str.split(',').values[0]
    sequences = df[df.audio == fnoext].sequences.str.split(',').values[0]

    tg = al.LabelManager(from_file = f, from_type='praat')

    # Make interval tier for target phonemes
    target_tier_seq = al.IntervalTier(name = 'targetseq')
    target_tier_phon = al.IntervalTier(name = 'targetphone')

    # Now search for word that matches
    for i in range(len(words)):
        w = words[i]

        matches = tg.tier('transcription - words').search(w)
        
        print(w, matches)
        print(sequences)

        # The match should be the one that matches the index
        for match in matches:

            # Search within times of target words
            phonlabs = tg.tier('transcription - phones').tslice(match.t1, match.t2, lincl=False, rincl=False)

            # Get times for phone sequence (ex. IH0 K AH0) within target words
            phontier = tg.tier('transcription - phones')

            # Sequence aimed for, in ARPAbet, and length
            seq = sequences[i]
            seq_arpa = seq_to_arpa[seq]
            seq_len = len(seq_arpa)

            for phonlab in phonlabs[:-(seq_len-1)]:

                seq_check = []

                for j in range(seq_len):
                    phontier.next(phonlab, skip = j-1).text
                    seq_check.append(phontier.next(phonlab, skip = j-1).text)

                # If the sequence being checked is correct
                if seq_check == seq_arpa:
                    # Add the sequence and phones
                    # Add t1 of phonlab, then t2 of phonlab 1 or 2 away, depending on length
                    target_tier_seq.add(al.Label(t1 = float(phonlab.t1), t2 = float(phontier.next(phonlab, skip = seq_len-2).t2), text = seq))

                    # Now add t1 and t2 of each phonlab
                    for j in range(seq_len):
                        phonlabel = phontier.next(phonlab, skip = j-1)
                        target_tier_phon.add(al.Label(t1 = float(phonlabel.t1), t2 = float(phonlabel.t2), text = phonlabel.text))

                    # Then break

                else:
                    next

    # # Add blank interval between time 0 and first annotation IF first annotation not edge
    # if target_tier_seq[0].t1 != 0: target_tier_seq.add(al.Label(t1 = float(0), t2 = float(target_tier_seq[0].t1), text = ''))
    # # Add blank interval btwn last annotation and end of file IF last not at edge
    # if target_tier_seq[-1].t2 != tg.tier(0)[-1].t2: target_tier_seq.add(al.Label(t1 = float(target_tier_seq[-1].t2), t2 = tg.tier(0)[-1].t2, text = ''))

    # #print(target_tier_seq)

    # Add blank interval between time 0 and first annotation IF first annotation not edge
    print(target_tier_seq)
    
    if target_tier_seq[0].t1 != 0: target_tier_seq.add(al.Label(t1 = float(0), t2 = float(target_tier_seq[0].t1), text = ''))
    # Add blank interval btwn last annotation and end of file IF last not at edge
    if target_tier_seq[-1].t2 != tg.tier(0)[-1].t2: target_tier_seq.add(al.Label(t1 = float(target_tier_seq[-1].t2), t2 = tg.tier(0)[-1].t2, text = ''))

    #print(target_tier_seq)

    # If there is more than one sequence, we need to also add blanks in between each annotation
    for interv in target_tier_seq[1:]:

        # Add blanks if not adjacent
        if target_tier_seq.prev(interv).t2 != interv.t1: 
            target_tier_seq.add(al.Label(t1 = float(target_tier_seq.prev(interv).t2), t2 = float(interv.t1), text = ''))

    ###
    # Repeat for phon
    ###

    # Add blank interval between time 0 and first annotation IF first annotation not edge
    if target_tier_phon[0].t1 != 0: target_tier_phon.add(al.Label(t1 = float(0), t2 = float(target_tier_phon[0].t1), text = ''))
    # Add blank interval btwn last annotation and end of file IF last not at edge
    if target_tier_phon[-1].t2 != tg.tier(0)[-1].t2: target_tier_phon.add(al.Label(t1 = float(target_tier_phon[-1].t2), t2 = tg.tier(0)[-1].t2, text = ''))

    # If there is more than one sequence, we need to also add blanks in between each annotation
    for interv in target_tier_phon[1:]:

        # Add blanks if not adjacent
        if target_tier_phon.prev(interv).t2 != interv.t1: 
            target_tier_phon.add(al.Label(t1 = float(target_tier_phon.prev(interv).t2), t2 = float(interv.t1), text = ''))

    tg.add(target_tier_seq, idx = 2)
    tg.add(target_tier_phon, idx = 3)

    return(tg)

In [13]:
folder = 'vowels'

# Get all files
files = [f for f in sorted(glob.glob('./VS/' + folder + '/files/*.TextGrid', recursive = True))]

# Now write target phone
for f in files:
    bname = os.path.basename(f).split('.')[0]
    
    print(f)
    
    tg = create_target_tiers(f, './VS/' + folder + '/targets.csv')
    
    # BASENAME: 
    fnew = './VS/' + folder + '/files_tgts/' + os.path.basename(f)
    
    # Write to TextGrid
    with open(fnew, 'w', encoding='utf-8') as w:
        w.write(tg.as_string('praat_long'))
    
    # Print extra line for readability
    print('\r')

./VS/vowels/files/VS-24_04.TextGrid
puqaapunmutihara [Label( t1=10.1200, t2=11.1700, text='b'puqaapunmutihara'' )]
['pu']
IntervalTier([Label( t1=10.1200, t2=10.2400, text='b'pu'' ),Label( t1=10.4600, t2=10.6300, text='b'pu'' )])


./VS/vowels/files/VS-24_11.TextGrid
kich [Label( t1=2.5000, t2=2.7600, text='b'kich'' )]
['ki']
IntervalTier([Label( t1=2.5000, t2=2.6100, text='b'ki'' )])


./VS/vowels/files/VS-24_12.TextGrid
kich [Label( t1=1.2800, t2=1.5600, text='b'kich'' )]
['ki']
IntervalTier([Label( t1=1.2800, t2=1.4200, text='b'ki'' )])


./VS/vowels/files/VS-33_39.TextGrid
xuntapan [Label( t1=3.3300, t2=4.3400, text='b'xuntapan'' )]
['xu']
IntervalTier([Label( t1=3.3300, t2=3.3900, text='b'xu'' )])


./VS/vowels/files/VS-34a_01.TextGrid
pufaat [Label( t1=0.5900, t2=1.3400, text='b'pufaat'' )]
['pu']
IntervalTier([Label( t1=0.5900, t2=0.7800, text='b'pu'' )])


./VS/vowels/files/VS-34a_02.TextGrid
pufaat [Label( t1=0.6200, t2=1.3700, text='b'pufaat'' )]
['pu']
IntervalTier([Label( t

### Orig

In [2]:
import audiolabel as al

In [3]:
# Dictionary mapping sequences to ARPABET to look for
seq_to_arpa = {
    r'u\d?xi': 'HH',
    r'u\d?fi': 'F',
    r'u\d?pi': 'P',
    r'i\d?ka': 'K',
    r'i\d?ku': 'K'
}

arpa_to_phon = {
    'UH0': 'u',
    'UW1': 'uu',
    'IH0': 'i',
    'IY1': 'ii',
    'AH0': 'a',
    'AA1': 'aa',
    'HH': 'x',
    'F': 'f',
    'P': 'p',
    'K': 'k'
}

In [4]:
def create_target_tiers(textgrid):
    '''
    Create tiers for sequences and for phonemes
    '''
    
    tg = al.LabelManager(from_file=textgrid, from_type='praat')

    # Make interval tier for target phonemes
    target_tier_seq = al.IntervalTier()
    target_tier_phon = al.IntervalTier()

    # Make list of intervals for which there should be labels
    intervals_seq = []
    intervals_phon = []

    # Search in word tier for target words with sequence
    for seq in seq_to_arpa.keys():

        matches = tg.tier('transcription - words').search(seq)

        if matches != []:

            # print(seq, matches)

            for match in matches:

                # Search within times of target words
                phonlabs = tg.tier('transcription - phones').tslice(match.t1, match.t2, lincl=False, rincl=False)

                # Vowel to left and right
                if seq_to_arpa[seq] == 'K':
                    left = ['IH0', 'IY1']
                    right = ['AH0', 'AA1', 'UH0', 'UW1']

                else:
                    left = ['UH0', 'UW1']
                    right = ['IH0', 'IY1']

                # Get times for phone sequence (ex. IH0 K AH0) within target words
                phontier = tg.tier('transcription - phones')

                for phonlab in phonlabs[2:]:

                    # If previous phone is the target consonant
                    if phontier.prev(phonlab).text == seq_to_arpa[seq]:

                        if (phontier.prev(phonlab, skip = 1).text in left) and (phonlab.text in right):
                            # Get data for VCV seq
                            pre_v = phontier.prev(phonlab, skip = 1)
                            c = phontier.prev(phonlab)
                            v = phonlab

                            #print(' '.join([pre_v.text, c.text, v.text]))
                            #print(''.join([arpa_to_phon[pre_v.text], arpa_to_phon[c.text], arpa_to_phon[v.text]]))

                            # Add the times for the sequences, then by phone
                            intervals_seq.append((pre_v.t1, v.t2))
                            target_tier_seq.add(al.Label(t1 = float(pre_v.t1), t2 = float(v.t2), text = ''.join([arpa_to_phon[pre_v.text], arpa_to_phon[c.text], arpa_to_phon[v.text]])))

                            for seg in [pre_v, c, v]:
                                intervals_phon.append((seg.t1, seg.t2))
                                target_tier_phon.add(al.Label(t1 = float(seg.t1), t2 = float(seg.t2), text = seg.text))

                            #print(intervals_seq, intervals_phon, target_tier_seq, target_tier_phon)
                            
    # Add blank interval between time 0 and first annotation IF first annotation not edge
    if target_tier_seq[0].t1 != 0: target_tier_seq.add(al.Label(t1 = float(0), t2 = float(target_tier_seq[0].t1), text = ''))
    # Add blank interval btwn last annotation and end of file IF last not at edge
    if target_tier_seq[-1].t2 != tg.tier(0)[-1].t2: target_tier_seq.add(al.Label(t1 = float(target_tier_seq[-1].t2), t2 = tg.tier(0)[-1].t2, text = ''))

    #print(target_tier_seq)

    # If there is more than one sequence, we need to also add blanks in between each annotation
    for interv in target_tier_seq[1:]:

        # Add blanks if not adjacent
        if target_tier_seq.prev(interv).t2 != interv.t1: 
            target_tier_seq.add(al.Label(t1 = float(target_tier_seq.prev(interv).t2), t2 = float(interv.t1), text = ''))

    ###
    # Repeat for phon
    ###

    # Add blank interval between time 0 and first annotation IF first annotation not edge
    if target_tier_phon[0].t1 != 0: target_tier_phon.add(al.Label(t1 = float(0), t2 = float(target_tier_phon[0].t1), text = ''))
    # Add blank interval btwn last annotation and end of file IF last not at edge
    if target_tier_phon[-1].t2 != tg.tier(0)[-1].t2: target_tier_phon.add(al.Label(t1 = float(target_tier_phon[-1].t2), t2 = tg.tier(0)[-1].t2, text = ''))

    # If there is more than one sequence, we need to also add blanks in between each annotation
    for interv in target_tier_phon[1:]:

        # Add blanks if not adjacent
        if target_tier_phon.prev(interv).t2 != interv.t1: 
            target_tier_phon.add(al.Label(t1 = float(target_tier_phon.prev(interv).t2), t2 = float(interv.t1), text = ''))

    tg.add(target_tier_seq, idx = 2)
    tg.add(target_tier_phon, idx = 3)
    
    return(tg)

In [5]:
#textgrid = 'VS/files/karuk-2010-05-01-arg-edited-2_42.TextGrid'

#create_target_tiers(textgrid)

# Get all files
files = [f for f in sorted(glob.glob('./VS/test/*.TextGrid', recursive = True))]

# Now write target phone
for f in files:
    bname = os.path.basename(f).split('.')[0]
    
    tg = create_target_tiers(f)
    
    # BASENAME: 
    f = './VS/files_tgts/' + os.path.basename(f)
    print(f)
    
    # Write to TextGrid
    with open(f, 'w', encoding='utf-8') as f:
        f.write(tg.as_string('praat_long'))
    
    # Print extra line for readability
    print('\r')

./VS/files_tgts/karuk-2010-07-23-ag-vs-4_37.TextGrid

